In [1]:
import numpy as np
from tensorflow.keras.preprocessing  import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,  BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
train_dir = r'hand\train'
validation_dir = r'hand\val'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255

)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size = 32,
    color_mode='grayscale',
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size = 32,
    color_mode='grayscale',
    class_mode='binary'
)


Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [5]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [6]:
history = model.fit(
   train_generator,
   steps_per_epoch=train_generator.samples // train_generator.batch_size,
   epochs=12,
   validation_data = validation_generator,
   validation_steps = validation_generator.samples // validation_generator.batch_size
)

Epoch 1/12
276/276 [==============================] - 316s 1s/step - loss: 1.0546 - accuracy: 0.5708 - val_loss: 1.9540 - val_accuracy: 0.5990
Epoch 2/12
276/276 [==============================] - 296s 1s/step - loss: 0.6629 - accuracy: 0.6077 - val_loss: 0.7925 - val_accuracy: 0.5885
Epoch 3/12
276/276 [==============================] - 306s 1s/step - loss: 0.6419 - accuracy: 0.6226 - val_loss: 1.3929 - val_accuracy: 0.7031
Epoch 4/12
276/276 [==============================] - 294s 1s/step - loss: 0.6306 - accuracy: 0.6409 - val_loss: 0.4817 - val_accuracy: 0.7569
Epoch 5/12
276/276 [==============================] - 297s 1s/step - loss: 0.6061 - accuracy: 0.6614 - val_loss: 1.1248 - val_accuracy: 0.6927
Epoch 6/12
276/276 [==============================] - 295s 1s/step - loss: 0.5971 - accuracy: 0.6720 - val_loss: 0.4460 - val_accuracy: 0.7934
Epoch 7/12
276/276 [==============================] - 294s 1s/step - loss: 0.5932 - accuracy: 0.6809 - val_loss: 0.6862 - val_accuracy: 0.4670

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy:{accuracy*100:.2f}%")

19/19 [==============================] - 4s 219ms/step - loss: 1.0138 - accuracy: 0.7383
Validation Accuracy:73.83%


In [12]:
print(train_generator.class_indices)

{'fractured': 0, 'not fractured': 1}


In [21]:
img_path = r'n.jpg'  
img = image.load_img(img_path, target_size=(150, 150), color_mode='grayscale')
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /=255.0

In [22]:
prediction = model.predict(img_array)
print(f"Raw prediction output: {prediction[0][0]}")

if prediction[0] < 0.5:
    print("The image is predicted to be fractured.")
else:
    print("The image is predicted to be not fractured.")

1/1 [==============================] - 0s 243ms/step
Raw prediction output: 0.027695752680301666
The image is predicted to be fractured.
